# Aula 06 (25/08/2023)

# Problema do Transporte

Suponha um problema onde o objetivo é minimizar o custo total do transporte de mercadorias entre as fábricas ou centros de distribuição e seus clientes.

<img src="ProblemaTransporte1.png" width="500" height="500">

Essa situação pode ser vista como um problema de programação linear, onde os dados são:  
- as capacidades das fábricas;  
- as demandas dos clientes;  
- $c_{ij}:$ capacidade de transporte de uma unidade de mercadoria da fábrica $i$ para o cliente $j$;  
- $X_{ij}:$ quantidade de mercadoria a ser transportada da fábrica $i$ para o cliente $j$.  

$\text{Maximizar}\ 2X_{13}+4X_{14}+5X_{15}+3X_{23}+2X_{24}+3X_{25}$  
$\text{Sujeito a}
    \begin{cases}
        \left. \begin{array}{l}
            X_{13}+X_{14}+X_{15} \le 200 \\
            X_{23}+X_{24}+X_{25} \le 250 \\
        \end{array}\right\} \text{Restrições de capacidade} \\
        \left. \begin{array}{l}
            X_{13}+X_{23} \ge 100 \\
            X_{14}+X_{24} \ge 200 \\
            X_{15}+X_{25} \ge 150 \\
        \end{array}\right\}\ \text{Restrições de demanda} \\
    X_{13},\ X_{14},\ X_{15},\ X_{23},\ X_{24},\ X_{25} \ge 0
    \end{cases}$  

## Exemplo (página 34 do livro)

Uma Vinícola tem a sua produção de vinhos em duas regiões do Brasil: Uma na região Nordeste, no Vale do São Francisco, que produz 700 mil litros e uma na região Sul, no Vale dos Vinhedos, que produz 950 mil litros. Essa vinícola transporta sua produção -ara 4 grandes distribuidores. No quadro abaixo encontram-se as informações de capacidade das indústrias, demandas dos distribuidoresm e custos de transporte. Modele o problema de forma que a vinícola gaste o mínimo oara transportar seus produtos até os drstribuidores.


| Produção | Dist. 1 | Dist. 2 | Dist. 3 | Dist. 4 | Capacidade (milhares de litros) |
|:---|:---:|:---:|:---:|:---:|:---:|
| Vale do São Francisco | 14 | 5 | 7 | 9 | 700.000 |
| Vale dos Vinhedos | 4 | 10 | 12 | 7 | 950.000 |
| Demanda (milhares de litros) | 450.000 | 380.000 | 400.000 | 420.000 | - |

*Solução:*

O problema acima tem a forma:

$\text{Minimizar}\ 14X_{13}+5X_{14}+7X_{15}+9X_{16}+X_{23}+10X_{24}+12X_{25}+7X_{26}$  
$\text{Sujeito a}
    \begin{cases}
        \left. \begin{array}{l}
            X_{13}+X_{14}+X_{15}+X_{16} \le 700000 \\
            X_{23}+X_{24}+X_{25}+X_{26} \le 950000 \\
        \end{array}\right\}\ \text{Restrições de capacidade} \\
        \left. \begin{array}{l}
            X_{13}+X_{23} \ge 450000 \\
            X_{14}+X_{24} \ge 380000 \\
            X_{15}+X_{25} \ge 400000 \\
            X_{16}+X_{26} \ge 420000 \\
        \end{array}\right\}\ \text{Restrições de demanda} \\
    X_{13},\ X_{14},\ X_{15},\ X_{16},\ X_{23},\ X_{24},\ X_{25},\ X_{26} \ge 0
\end{cases}$  

In [5]:
# Bibliotecas a serem usadas
using JuMP, GLPK

In [12]:
# Custos
c=[14, 5, 7, 9, 4, 10, 12, 7]
# Capacidades e demandas
b=[700000, 950000, 450000, 380000, 400000, 420000]

# Índices das variáveis
start_node = [1,1,1,1,2,2,2,2]
end_node = [3,4,5,6,3,4,5,6]

links = Tuple( (start_node[i], end_node[i]) for i in 1:length(end_node) )
c_dict = Dict(links .=> c)
print(links)

((1, 3), (1, 4), (1, 5), (1, 6), (2, 3), (2, 4), (2, 5), (2, 6))

In [15]:
# Modelo
model = Model(GLPK.Optimizer)

# Variáveis
@variable(model, x[link in links]>=0)

# Função objetivo
@objective(model, Min, sum(c_dict[link] * x[link] for link in links))

# Restrições de capacidade
for i in 1:2
   @constraint(model, sum(x[(ii,j)] for (ii,j) in links if ii==i) == b[i])
end
# Restrições de demandas
for j in 3:6
   @constraint(model, sum(x[(i,jj)] for (i,jj) in links if jj==j) <= b[j])
end

# Descrição do  modelo
println(model)

Min 14 x[(1, 3)] + 5 x[(1, 4)] + 7 x[(1, 5)] + 9 x[(1, 6)] + 4 x[(2, 3)] + 10 x[(2, 4)] + 12 x[(2, 5)] + 7 x[(2, 6)]
Subject to
 x[(1, 3)] + x[(1, 4)] + x[(1, 5)] + x[(1, 6)] == 700000
 x[(2, 3)] + x[(2, 4)] + x[(2, 5)] + x[(2, 6)] == 950000
 x[(1, 3)] + x[(2, 3)] <= 450000
 x[(1, 4)] + x[(2, 4)] <= 380000
 x[(1, 5)] + x[(2, 5)] <= 400000
 x[(1, 6)] + x[(2, 6)] <= 420000
 x[(1, 3)] >= 0
 x[(1, 4)] >= 0
 x[(1, 5)] >= 0
 x[(1, 6)] >= 0
 x[(2, 3)] >= 0
 x[(2, 4)] >= 0
 x[(2, 5)] >= 0
 x[(2, 6)] >= 0



In [16]:
# Otimização do modelo
optimize!(model)
obj = objective_value(model)

# Resultados
println("Função Objetivo: FO=", obj)
for link in links
  println("Valor de x",link," = ",value(x[link]))
end

Função Objetivo: FO=9.84e6
Valor de x(1, 3) = 0.0
Valor de x(1, 4) = 380000.0
Valor de x(1, 5) = 320000.0
Valor de x(1, 6) = 0.0
Valor de x(2, 3) = 450000.0
Valor de x(2, 4) = 0.0
Valor de x(2, 5) = 80000.0
Valor de x(2, 6) = 420000.0


# Problema desbalanceado

No caso de um problema desbalanceado, isto é, existe diferença entre a soma das capacidades e a soma das demandas, o problema não terá solução. Uma maneira de contornar esse situação é a criação de nós *dummy* de capacidade ou demanda de custo elevado para conter o excesso.

## Exemplo

<img src="ProblemaTransporte2.png" width="500" height="500">

$\text{Minimizar}\ X_{13}+3X_{14} \color{purple}{+20X_{15}}\color{black}{+5X_{23}+2X_{24}}\color{purple}{+10X_{25}}$  
$\text{Sujeito a}
    \begin{cases}
        \left. \begin{array}{l}
            X_{13}+X_{14}\color{purple}{+X_{15}}\color{purple}{\le 250}\\
            X_{23}+X_{24}\color{purple}{+X_{25}}\color{purple}{\le 500}\\
        \end{array}\right\} \text{Restrições de capacidade} \\
        \left. \begin{array}{l}
            X_{13}+X_{23} \ge 300 \\
            X_{14}+X_{24} \ge 150 \\
            \color{purple}{X_{15}+X_{25} \ge 300} \\
        \end{array}\right\}\ \text{Restrições de demanda} \\
    X_{13},\ 4X_{14},\ \color{purple}{X_{15},}\ \color{black}{X_{23},\ X_{24},}\ \color{purple}{X_{25}} \color{black}{\ge 0}
\end{cases}$ 

In [17]:
# Custos
c=[1, 3, 20, 5, 2, 10]
# Capacidades e demandas
b=[250, 500, 300, 150, 300]

# Índices das variáveis
start_node = [1,1,1,2,2,2]
end_node = [3,4,5,3,4,5]

links = Tuple( (start_node[i], end_node[i]) for i in 1:length(end_node) )
c_dict = Dict(links .=> c)
print(links)

((1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5))

In [19]:
# Modelo
model = Model(GLPK.Optimizer)

# Variáveis
@variable(model, x[link in links]>=0)

# Função objetivo
@objective(model, Min, sum(c_dict[link] * x[link] for link in links))

# Restrições de capacidade
for i in 1:2
   @constraint(model, sum(x[(ii,j)] for (ii,j) in links if ii==i) == b[i])
end
# Restrições de demandas
for j in 3:5
   @constraint(model, sum(x[(i,jj)] for (i,jj) in links if jj==j) <= b[j])
end

# Descrição do  modelo
println(model)

Min x[(1, 3)] + 3 x[(1, 4)] + 20 x[(1, 5)] + 5 x[(2, 3)] + 2 x[(2, 4)] + 10 x[(2, 5)]
Subject to
 x[(1, 3)] + x[(1, 4)] + x[(1, 5)] == 250
 x[(2, 3)] + x[(2, 4)] + x[(2, 5)] == 500
 x[(1, 3)] + x[(2, 3)] <= 300
 x[(1, 4)] + x[(2, 4)] <= 150
 x[(1, 5)] + x[(2, 5)] <= 300
 x[(1, 3)] >= 0
 x[(1, 4)] >= 0
 x[(1, 5)] >= 0
 x[(2, 3)] >= 0
 x[(2, 4)] >= 0
 x[(2, 5)] >= 0



In [20]:
# Otimização do modelo
optimize!(model)
obj = objective_value(model)

# Resultados
println("Função Objetivo: FO=", obj)
for link in links
  println("Valor de x",link," = ",value(x[link]))
end

Função Objetivo: FO=3800.0
Valor de x(1, 3) = 250.0
Valor de x(1, 4) = 0.0
Valor de x(1, 5) = 0.0
Valor de x(2, 3) = 50.0
Valor de x(2, 4) = 150.0
Valor de x(2, 5) = 300.0


# Exercícios

Coloque os exercícios abaixo no modelo do problema de transportes utilizando a linguagem Julia e resolva-os.

**1.** Uma empresa fabricante de bicicletas possui 3 fábricas localizadas no Rio, São Paulo e Belo Horizonte. A produção deve ser entregue em Recife, Salvador e Manaus. Considerando os custos de transporte unitários, as capacidades de produção e as demandas dos centros consumidores que estão especificados na tabela a seguir, determine quanto deve ser produzido e entregue por cada fábrica em cada centro consumidor de forma a minimizar os custos de transporte.

| | Centro de Recife | Centro Salvador | Centro Manaus | Capacidade |
|:---:|:---:|:---:|:---:|:---:|
| Rio | 25 | 20 | 30 | 2000 |
| São Paulo | 30 | 25 | 25 | 1500 |
| Belo Horizonte | 20 | 15 | 3 | 1500 |
| Demanda | 2000 | 2000 | 2000 | 5000 |

*Solução:*

In [23]:
# Custos
c=[25, 20, 30, 30, 25, 25, 20, 15, 23]
# Capacidades e demandas
b=[2000, 1500, 1200, 2000, 2000, 1000]

start_node = [1,1,1,2,2,2,3,3,3]
end_node = [4,5,6,4,5,6,4,5,6]

# Índices das variáveis
links = Tuple( (start_node[i], end_node[i]) for i in 1:length(end_node) )
c_dict = Dict(links .=> c)
print(links)

((1, 4), (1, 5), (1, 6), (2, 4), (2, 5), (2, 6), (3, 4), (3, 5), (3, 6))

In [25]:
# Modelo
model = Model(GLPK.Optimizer)

# Variáveis
@variable(model, x[link in links]>=0)

# Função objetivo
@objective(model, Min, sum(c_dict[link] * x[link] for link in links))

# Restrições de capacidade
for i in 1:3 
   @constraint(model, sum(x[(ii,j)] for (ii,j) in links if ii==i) == b[i])
end
# Restrições de capacidade
for j in 4:6 
   @constraint(model, sum(x[(i,jj)] for (i,jj) in links if jj==j) <= b[j])
end

# Descrição do  modelo
println(model)

Min 25 x[(1, 4)] + 20 x[(1, 5)] + 30 x[(1, 6)] + 30 x[(2, 4)] + 25 x[(2, 5)] + 25 x[(2, 6)] + 20 x[(3, 4)] + 15 x[(3, 5)] + 23 x[(3, 6)]
Subject to
 x[(1, 4)] + x[(1, 5)] + x[(1, 6)] == 2000
 x[(2, 4)] + x[(2, 5)] + x[(2, 6)] == 1500
 x[(3, 4)] + x[(3, 5)] + x[(3, 6)] == 1200
 x[(1, 4)] + x[(2, 4)] + x[(3, 4)] <= 2000
 x[(1, 5)] + x[(2, 5)] + x[(3, 5)] <= 2000
 x[(1, 6)] + x[(2, 6)] + x[(3, 6)] <= 1000
 x[(1, 4)] >= 0
 x[(1, 5)] >= 0
 x[(1, 6)] >= 0
 x[(2, 4)] >= 0
 x[(2, 5)] >= 0
 x[(2, 6)] >= 0
 x[(3, 4)] >= 0
 x[(3, 5)] >= 0
 x[(3, 6)] >= 0



In [26]:
# Otimização do modelo
optimize!(model)
obj = objective_value(model)

# Resultados
println("Função Objetivo: FO=", obj)
for link in links
  println("Valor de x",link," = ",value(x[link]))
end

Função Objetivo: FO=104000.0
Valor de x(1, 4) = 1700.0
Valor de x(1, 5) = 300.0
Valor de x(1, 6) = 0.0
Valor de x(2, 4) = 0.0
Valor de x(2, 5) = 500.0
Valor de x(2, 6) = 1000.0
Valor de x(3, 4) = 0.0
Valor de x(3, 5) = 1200.0
Valor de x(3, 6) = 0.0


**2.** Miss Daisy Ltda é um laboratório de manipulação que presta serviços de entrega para idosos. A empresa possui duas filiais e fornece o serviço a seis bairros diferentes. As capacidades das filiais, as demandas dos bairros e os custos unitários de entrega estão evidenciados na tabela a seguir. Quais clientes atender, a partir de cada filial, de maneira a minimizar o seu custo de entrega?

| | Ipanema | Copacabana | Centro | Barra | Leblon | Tijuca | capacidade |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| Filial Centro | 7,00 | 9,00 | 1,00 | 12,00 | 7,00 | 4,00 | 2500,00 |
| Filial Barra | 4,00 | 5,00 | 12,00 | 1,00 | 3,00 | 8,00 | 2000,00 |
| Demanda | 1400,00 | 1560,00 | 300,00 | 150,00 | 570,00 | 520,00 | 4500,00 |

*Solução:*

Temos um problema do transporte onde 2 filiais suprem a demanda de 6 bairros.

In [32]:
# Custos
c=[7, 9, 1, 12, 7, 4, 4, 5, 12, 1, 3, 8]
# Capacidades e demandas
b=[2500, 2000, 1400, 1560, 300, 1250, 570, 520]

# Índices das variáveis
start_node = [1,1,1,1,1,1,2,2,2,2,2,2]
end_node = [3,4,5,6,7,8,3,4,5,6,7,8]

links = Tuple( (start_node[i], end_node[i]) for i in 1:length(end_node) )
c_dict = Dict(links .=> c)
print(links)

((1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8))

In [33]:
# Modelo
model = Model(GLPK.Optimizer)

# Variáveis
@variable(model, x[link in links]>=0)

# Função objetivo
@objective(model, Min, sum(c_dict[link] * x[link] for link in links))

# Restrições de capacidade
for i in 1:2
   @constraint(model, sum(x[(ii,j)] for (ii,j) in links if ii==i) == b[i])
end
# Restrições de capacidade
for j in 3:8 
   @constraint(model, sum(x[(i,jj)] for (i,jj) in links if jj==j) <= b[j])
end

# Descrição do  modelo
println(model)

Min 7 x[(1, 3)] + 9 x[(1, 4)] + x[(1, 5)] + 12 x[(1, 6)] + 7 x[(1, 7)] + 4 x[(1, 8)] + 4 x[(2, 3)] + 5 x[(2, 4)] + 12 x[(2, 5)] + x[(2, 6)] + 3 x[(2, 7)] + 8 x[(2, 8)]
Subject to
 x[(1, 3)] + x[(1, 4)] + x[(1, 5)] + x[(1, 6)] + x[(1, 7)] + x[(1, 8)] == 2500
 x[(2, 3)] + x[(2, 4)] + x[(2, 5)] + x[(2, 6)] + x[(2, 7)] + x[(2, 8)] == 2000
 x[(1, 3)] + x[(2, 3)] <= 1400
 x[(1, 4)] + x[(2, 4)] <= 1560
 x[(1, 5)] + x[(2, 5)] <= 300
 x[(1, 6)] + x[(2, 6)] <= 1250
 x[(1, 7)] + x[(2, 7)] <= 570
 x[(1, 8)] + x[(2, 8)] <= 520
 x[(1, 3)] >= 0
 x[(1, 4)] >= 0
 x[(1, 5)] >= 0
 x[(1, 6)] >= 0
 x[(1, 7)] >= 0
 x[(1, 8)] >= 0
 x[(2, 3)] >= 0
 x[(2, 4)] >= 0
 x[(2, 5)] >= 0
 x[(2, 6)] >= 0
 x[(2, 7)] >= 0
 x[(2, 8)] >= 0



In [34]:
# Otimização do modelo
optimize!(model)
obj = objective_value(model)

# Resultados
println("Função Objetivo: FO=", obj)
for link in links
  println("Valor de x",link," = ",value(x[link]))
end

Função Objetivo: FO=18560.0
Valor de x(1, 3) = 1400.0
Valor de x(1, 4) = 0.0
Valor de x(1, 5) = 300.0
Valor de x(1, 6) = 0.0
Valor de x(1, 7) = 280.0
Valor de x(1, 8) = 520.0
Valor de x(2, 3) = 0.0
Valor de x(2, 4) = 460.0
Valor de x(2, 5) = 0.0
Valor de x(2, 6) = 1250.0
Valor de x(2, 7) = 290.0
Valor de x(2, 8) = 0.0
